# Continuous Answer Type

This notebook demonstrates how to generate forecasting questions with numeric/continuous answers. Continuous questions are perfect for quantitative predictions like sports statistics, financial metrics, or any numeric outcome.

In [5]:
%pip install -e ..
%pip install python-dotenv

from IPython.display import clear_output
clear_output()

## Set up the client

**Set your API key:**

- **Locally**: Create a `.env` file in this directory with `LIGHTNINGROD_API_KEY=your_key_here`
- **Google Colab**: Go to the Secrets section (🔑 icon in left sidebar) and add a secret named `LIGHTNINGROD_API_KEY`

In [6]:
from utils import get_secret
from lightningrod import LightningRod

api_key = get_secret("LIGHTNINGROD_API_KEY")
base_url = get_secret("LIGHTNINGROD_BASE_URL", "https://api.lightningrod.ai/api/public/v1")

lr = LightningRod(api_key=api_key, base_url=base_url)

## Continuous Answer Type Overview

Continuous questions return numeric values. They're ideal for:
- Sports statistics (points, rebounds, assists)
- Financial predictions (stock prices, revenue)
- Metrics forecasting (user growth, sales numbers)
- Any quantitative outcome

Continuous questions work with any data source: News Search, Top Aggregated News, or Custom Documents.

In [7]:
from datetime import datetime, timedelta
from lightningrod import AnswerType, AnswerTypeEnum, NewsSeedGenerator, QuestionGenerator, QuestionPipeline, WebSearchLabeler, QuestionRenderer

answer_type = AnswerType(answer_type=AnswerTypeEnum.CONTINUOUS)

seed_generator = NewsSeedGenerator(
    start_date=datetime.now() - timedelta(days=30),
    end_date=datetime.now(),
    search_query=[
        "NBA player statistics",
        "NBA player performance",
        "NBA game statistics",
    ],
)

question_generator = QuestionGenerator(
    instructions=(
        "Write forecasting questions that require a single, specific numeric answer. "
        "Avoid questions that could be answered with a range or category—ask about concrete statistics or totals. "
        "Make sure the question is clear, unambiguous, and focused on a quantifiable outcome occurring in the future."
    ),
    examples=[
        "How many points will LeBron James score in his next game?",
        "How many rebounds will the Lakers grab in their next game?",
        "How many points will the Bulls score in their next game?",
        "How many three-pointers will Steph Curry make in his next game?",
    ],
    bad_examples=[
        "Will LeBron James score more than 30 points in his next game?",         # Binary, not a continuous answer
        "Which team will win the next Lakers game?",                             # Categorical, not numeric
        "How likely is it that the Bulls will score over 100 points?",           # Probability, not a specific number
        "How many rebounds did the Lakers grab in their last game?",             # Backward-looking, not a forecast
        "How has LeBron's scoring average changed this season?",                 # Vague, not a concrete forecast
    ],
    answer_type=answer_type,
)

# Labeler automatically finds answers to questions using web search
labeler = WebSearchLabeler(
    answer_type=answer_type,
    confidence_threshold=0.5,
)

pipeline_config = QuestionPipeline(
    seed_generator=seed_generator,
    question_generator=question_generator,
    labeler=labeler,
    renderer=QuestionRenderer(answer_type=answer_type),
)

## Run the Pipeline

Generate continuous questions from the news articles.

In [8]:
dataset = lr.transforms.run(pipeline_config, max_questions=10) # keep max questions low when testing

> Note: This can take a few minutes to complete processing.

## View Results

Inspect the generated questions and answers. Each sample contains `seed`, `question`, `label`, `prompt`, and optional `context` and `meta` fields. See [API.md](../API.md) for the complete sample structure.

In [9]:
%pip install pandas

from IPython.display import clear_output
clear_output()

In [10]:
import pandas as pd

# Download samples to memory
samples = dataset.download()
print(f"Generated {dataset.num_rows} samples\n")

# Convert cached samples to a list of dictionaries
rows = dataset.flattened()

df = pd.DataFrame(rows)
df

Generated 10 samples



,question.question_text,label.label,label.label_confidence,label.resolution_date,label.reasoning,prompt,seed.seed_text,seed.url,seed.seed_creation_date,seed.search_query,is_valid,meta.sample_id,meta.parent_sample_id,meta.processing_time_ms,meta.filter_reason,label.answer_sources
0,How many points will the Golden State Warriors...,118,1.0,2026-01-05T00:00:00,A Google search for 'Golden State Warriors vs ...,QUESTION:\nHow many points will the Golden Sta...,Title: Golden State Warriors vs LA Clippers Ja...,https://www.nba.com/game/gsw-vs-lac-0022500508...,2026-01-07T00:00:00,NBA player statistics,True,64af892f-2530-404c-b6e1-650948105766,c1a3e287-7ed8-4fdb-bb34-f582d5dbe1b8,10397.322,NaN,NaN
1,How many points will Emersynn Cage score in he...,undetermined,1.0,NaN,The question asks for the number of points Eme...,NaN,Title: WMass Daily Girls Basketball Stats Lead...,https://www.masslive.com/highschoolsports/2026...,2026-01-07T00:00:00,NBA player statistics,False,59a11219-6655-4f50-b285-e3b124401ede,af44ae5e-62ea-40cc-baea-528169370f9f,4458.911,Undetermined label,NaN
2,How many points will Jordan Williams (Deptford...,undetermined,1.0,NaN,The question asks for the exact number of poin...,NaN,Title: Boys Basketball: Tri-County Conference ...,https://www.nj.com/highschoolsports/2026/01/bo...,2026-01-07T00:00:00,NBA player statistics,False,7e6cf50b-ca72-4bc9-9ff5-dd5be236658b,10f20a46-c47a-4ad0-8d04-299c962994f5,6493.240,Undetermined label,https://vertexaisearch.cloud.google.com/ground...
3,How many total points will the UConn Huskies s...,undetermined,1.0,NaN,The question asks for the total points scored ...,NaN,Title: Providence men's basketball vs. UConn: ...,https://www.providencejournal.com/live-story/s...,2026-01-07T00:00:00,NBA player statistics,False,728dd1cb-5218-4bef-9792-1ac68d9d2574,ac7ae952-7f6b-44ce-b22d-743c22adf7d7,16672.270,Undetermined label,https://vertexaisearch.cloud.google.com/ground...
4,How many points will Kawhi Leonard score in hi...,31,1.0,2026-01-11T00:00:00,To determine the number of points Kawhi Leonar...,QUESTION:\nHow many points will Kawhi Leonard ...,Title: Leonard impresses in fifth NBA 2K26 rat...,https://www.espn.com/gaming/story/_/id/4756137...,2026-01-07T00:00:00,NBA player statistics,True,69014db2-763c-4249-ad85-81f0391db392,460ffc94-e531-4fb1-904e-3c3a5f8dc4c2,12700.225,NaN,https://vertexaisearch.cloud.google.com/ground...
5,How many points will Shai Gilgeous-Alexander s...,undetermined,1.0,NaN,The question asks for the number of points Sha...,NaN,Title: Is Shai Gilgeous-Alexander the most clu...,https://www.oklahoman.com/story/sports/nba/thu...,2026-01-07T00:00:00,NBA player statistics,False,1d68b2ae-72a0-4598-a612-bdcf08d6213d,5ee611de-098f-49d6-a5c7-9af06c96738f,3608.271,Undetermined label,NaN
6,How many points will Lamar Wilkerson score in ...,undetermined,1.0,NaN,A search for 'Lamar Wilkerson Indiana basketba...,NaN,Title: Three reasons Indiana men's basketball ...,https://www.thedailyhoosier.com/three-reasons-...,2026-01-14T00:00:00,NBA player statistics,False,bfd899f1-670e-4f7f-aa6f-b25cd94182be,7a8cce39-ad18-4b73-87c4-e85afab8c90b,18848.011,Undetermined label,https://vertexaisearch.cloud.google.com/ground...
7,How many points will JoJo Newell score in his ...,undetermined,1.0,NaN,I am unable to provide an answer to this quest...,NaN,Title: Boys Basketball: Shore Conference stats...,https://www.nj.com/highschoolsports/2026/01/bo...,2026-01-07T00:00:00,NBA player statistics,False,852934c5-d163-463e-93fa-ae717fc2aefc,b73eae72-a61c-48be-bf56-3b7f703252fe,11381.318,Undetermined label,https://vertexaisearch.cloud.google.com/ground...
8,How many points will the New York Knicks score...,undetermined,1.0,NaN,The game between the New York Knicks and the T...,NaN,Title: Intro to Gravity\n\nURL Source: https:/...,https://www.nba.com/news/intro-to-gravity-stat...,2026-01-07T00:00:00,NBA player statistics,False,8f9c3838-9c83-4194-b4b6-713e0f267259,efbf94b0-f9f0-45dd-af77-2af6e8555324,7065.098,Undete

## Use Cases for Continuous Questions

**Sports Statistics:**
- "How many points will Player X score?"
- "How many rebounds will Team Y grab?"

**Financial Predictions:**
- "What will Stock X's price be?"
- "What will Company Y's revenue be?"

**Metrics Forecasting:**
- "How many users will the app have?"
- "What will the conversion rate be?"